In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt

def pixelate_image(image, pixel_size):
    """
    Pixelates a given image array to the specified pixel size.
    """
    h, w = image.shape[:2]
    temp = cv2.resize(image, (pixel_size, pixel_size), interpolation=cv2.INTER_LINEAR)
    pixelated = cv2.resize(temp, (w, h), interpolation=cv2.INTER_NEAREST)
    return pixelated

def process_directory(input_dir, output_dir, pixel_size=45, visualize=False):
    """
    Applies pixelation to all images in the input_dir and saves results to output_dir.
    If visualize is True, displays the original, cropped, and pixelated images.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for root, dirs, files in os.walk(input_dir):
        rel_path = os.path.relpath(root, input_dir)
        out_subdir = os.path.join(output_dir, rel_path)
        if not os.path.exists(out_subdir):
            os.makedirs(out_subdir)
        for file in files:
            input_path = os.path.join(root, file)
            output_path = os.path.join(out_subdir, file)
            try:
                with Image.open(input_path) as img:
                    img_rgb = img.convert('RGB')
                    img_array = cv2.cvtColor(np.array(img_rgb), cv2.COLOR_RGB2BGR)
                    # Pixelate the whole face image (face cropping is already done)
                    pixelated = pixelate_image(img_array, pixel_size)
                    # Save pixelated image
                    pixelated_pil = Image.fromarray(cv2.cvtColor(pixelated, cv2.COLOR_BGR2RGB))
                    pixelated_pil.save(output_path, format='PNG')
                    print(f"Pixelated and saved {output_path}")

                    if visualize:
                        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
                        axes[0].imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
                        axes[0].set_title('Cropped Face')
                        axes[0].axis('off')
                        axes[1].imshow(cv2.cvtColor(pixelated, cv2.COLOR_BGR2RGB))
                        axes[1].set_title(f'Pixelated Face ({pixel_size}x{pixel_size})')
                        axes[1].axis('off')
                        plt.tight_layout()
                        plt.show()
            except UnidentifiedImageError:
                print(f"Could not identify image file {file}. Skipping...")
            except Exception as e:
                print(f"Error processing {file}: {e}")

if __name__ == "__main__":
    input_dir = './demographic_group1_faces_v2'    # Directory with cropped faces
    output_dir = './demographic_group1_faces_pixelated'  # Directory for pixelated faces
    pixel_size = 45  # Adjust the pixelation size as needed
    process_directory(input_dir, output_dir, pixel_size=pixel_size, visualize=False)